# Partiendo de los avances de Nerea

Se ha parametrizado:<br>
>timestep<br>
>valProportion: _para definir que % se deja para validación_<br>
>usedCols: _para seleccionar las columnas_<br>
>n_features: _se calcula partiendo de usedCols. La cantidad de columnas._<br>

<div class="alert alert-block alert-info">
Inspirado en:
__[Multivariate Time Series Forecasting with LSTMs in Keras](https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/)__
</div>


In [207]:
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.layers import TimeDistributed

# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    print(data.shape[1])
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg



In [208]:
timestep=10
valProportion=0.05
usedCols=["LME_PR_AL_serie_ROC_7", "LME_PR_AL_serie_MOM_15", "LME_PR_AL_serie_MOM_30", "LME_PR_AL_serie_close", "OPEC_ORB_serie_ROC_7", "OPEC_ORB_serie_MOM_30", "OPEC_ORB_serie_ROC_30", "OPEC_ORB_serie_ROC_15"]
n_features = len(usedCols)
print("n_features --> " + str(n_features))

n_features --> 8


In [209]:
# load dataset
dataset = read_csv('data/data4.csv',usecols=usedCols)
values = dataset.values
reframed =  series_to_supervised(values,timestep, 1)
# ensure all data is float
values = reframed.astype('float32')

8


In [210]:
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
print(scaled.shape)
#print(scaled[0])
#print(scaled[1])


(842, 88)


In [213]:
# specify the number of lag hours


# split into train and test sets
#values = reframed.values
#print(values.shape)
n_train_hours = round(scaled.shape[0] * (1-valProportion))
train = scaled[:n_train_hours, :]
test = scaled[n_train_hours:, :]
print("train")
print (train.shape)
print("test")
print (test.shape)



# split into input and outputs
#n_obs = 1000
# n_obs = n_hours

#print (train[:, -n_features])
train_X, train_y = train[:, :n_features*timestep], train[:, -n_features]
test_X, test_y = test[:, :n_features*timestep], test[:, -n_features]
print(train_X.shape, len(train_X), train_y.shape)



# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], timestep, n_features))
test_X = test_X.reshape((test_X.shape[0], timestep, n_features))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)


# design network
#model = Sequential()
#model.add(LSTM(100, input_shape=(train_X.shape[1], train_X.shape[2])))
#model.add(Dense(n_features))
#model.compile(loss='mae', optimizer='adam', metrics=['accuracy'])
#model.summary()

model = Sequential()
model.add(LSTM(timestep, input_shape=(timestep, 8), return_sequences=True))
model.add(TimeDistributed(Dense(timestep)))
model.compile(loss='mean_squared_error', optimizer='adam')
print(model.summary())




train
(800, 88)
test
(42, 88)
(800, 80) 800 (800,)
(800, 10, 8) (800,) (42, 10, 8) (42,)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_37 (LSTM)               (None, 10, 10)            760       
_________________________________________________________________
time_distributed_4 (TimeDist (None, 10, 10)            110       
Total params: 870
Trainable params: 870
Non-trainable params: 0
_________________________________________________________________
None


In [214]:
# fit network
history = model.fit(train_X, train_y, epochs=20, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)

# plot history
pyplot.plot(history.history['loss'], label='train loss')
pyplot.plot(history.history['val_loss'], label='val loss')
pyplot.legend()
pyplot.show()
# plot history
pyplot.plot(history.history['acc'], label='train acc')
pyplot.plot(history.history['val_acc'], label='val acc')
pyplot.legend()
pyplot.show()


ValueError: Error when checking target: expected time_distributed_4 to have 3 dimensions, but got array with shape (800, 1)

In [ ]:
testing=[1,2,3,4,5,6,7,8,9]
testing[-7:]

In [ ]:
print(test_X.shape)
test_X = test_X.reshape((test_X.shape[0], timestep, n_features))
print(test_X.shape)
# make a prediction
yhat = model.predict(test_X)
print("yhat.shape -->" + str(yhat.shape))
test_X = test_X.reshape((test_X.shape[0], timestep*n_features))
print(test_X.shape)

# invert scaling for forecast
print(yhat.shape)
inv_yhat = concatenate((yhat, test_X[:, -n_features:]), axis=1)
print("inv_hat")
print(inv_yhat)

inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]
#invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, test_X[:, -7:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]
# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
#rmse = sqrt(mean_squared_error(yhat, test_X[:, -1:]))
print('Test RMSE: %.3f' % rmse)